In [1]:
import json
import csv

with open('suomi24.json') as data_file:    
    data = json.load(data_file)

In [3]:
data[0].keys()

dict_keys(['title', 'created_at', 'views', 'topics', 'closed', 'thread_id', 'anonnick', 'comments', 'body', 'deleted'])

In [4]:
len(data)

54885

In [9]:
data[8]['topics'][0]['title']

'Nuoret'

In [25]:
for i in range(100,200):
    print(i)
    print(data[i]['topics'])
    print(data[i]['body'])

100
[{'title': 'Terveys', 'topic_id': 4}, {'title': 'Päihteet ja riippuvuudet', 'topic_id': 2123}, {'title': 'Alkoholi', 'topic_id': 3650}]
<p>,Torstaita tänään ei ruohokaan kasva.</p><p>Kirjoitellaan nyt jotain sitte,,,</p><p>Kyllä minä olen raamatun lukenut se on ihan mielenkiintoinen ja vaikken nyt ole kovin uskonnollinen niin olen kiinnostunut eri uskonnoista ja mytologiasta. Liian kova aihe suomi24:lle ja en ole väitöskirjaa tekemässä,,,sellainen harrastus vain.</p><p>Olen eronnut kirkosta ihan vaan sillä syyllä etten voi kannattaa nykyistä kaupallista sanotaanko omituisesti vaikkapa,,, anetoimintaa ja tuota ette välttämättä käsitä koska en minäkään täysin.</p><p>Mulla nauratti ihan penteleesti kun on ollut näitä buumeja jolloin erottiin kirkosta vain siksi kun mediassa on joku Päikkä Räsäsen haastattelu tai muu väittely tasa-arvoisen avioliittolain,,, ? siis vidun jahkausta sähän voit kuulua kirkkoon vain omasta syystäsi tai olla kuulumatta ja haistattaa vidun muiden mielipiteill

In [26]:
counts = {}
for d in data:
    topic = d['topics'][0]['title']
    if not topic in counts: counts[topic] = 0
    counts[topic] += 1
counts

{'Ajanviete': 816,
 'Ajoneuvot ja liikenne': 2818,
 'Harrastukset': 1509,
 'Koti ja rakentaminen': 1127,
 'Lemmikit': 725,
 'MainPage': 3,
 'Matkailu': 984,
 'Muoti ja kauneus': 364,
 'Nuoret': 906,
 'Paikkakunnat': 8766,
 'Perhe': 507,
 'Ruoka ja juoma': 290,
 'Ryhmät': 1104,
 'Suhteet': 6790,
 'Suomi24': 126,
 'Talous': 1348,
 'Terveys': 2302,
 'Tiede ja teknologia': 2165,
 'Tori': 147,
 'Työ ja opiskelu': 1137,
 'Urheilu ja kuntoilu': 910,
 'Viihde ja kulttuuri': 1797,
 'Yhteiskunta': 18244}

In [29]:
topics = ['Ajanviete',
 'Ajoneuvot ja liikenne',
 'Harrastukset',
 'Koti ja rakentaminen',
 'Lemmikit',
 'MainPage',
 'Matkailu',
 'Muoti ja kauneus',
 'Nuoret',
 'Paikkakunnat',
 'Perhe',
 'Ruoka ja juoma',
 'Ryhmät',
 'Suhteet',
 'Suomi24',
 'Talous',
 'Terveys',
 'Tiede ja teknologia',
 'Tori',
 'Työ ja opiskelu',
 'Urheilu ja kuntoilu',
 'Viihde ja kulttuuri',
 'Yhteiskunta']

In [32]:
topics = [ 'Ajoneuvot ja liikenne',
 'Koti ja rakentaminen',
 'Paikkakunnat',
 'Suhteet',
 'Terveys',
 'Yhteiskunta' ]

In [40]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [41]:
strip_tags('<p>Suomen talous on kuralla ja siihen on yksi syyllinen: ammattiyhdistysliike.</p><p>Vielä muutama vuosi sitten voimakasta ammattiyhdistysliikettä pidettiin suomalaisen yhteiskunnan vahvuutena. Ihailtiin konsensusta, sopimisen kulttuuria ja muuta työmarkkinanostalgiaa</p><p>Juuri')

'Suomen talous on kuralla ja siihen on yksi syyllinen: ammattiyhdistysliike. Vielä muutama vuosi sitten voimakasta ammattiyhdistysliikettä pidettiin suomalaisen yhteiskunnan vahvuutena. Ihailtiin konsensusta, sopimisen kulttuuria ja muuta työmarkkinanostalgiaa Juuri'

In [42]:
rows = []
rows.append(['id','text'] + topics)
for idx,d in enumerate(data):
    d_topic = d['topics'][0]['title']
    if not d_topic in topics: continue
    row = [idx,strip_tags(d['body'])]
    topic_row = []
    for t in topics:
        if d_topic == t: 
            topic_row.append(1)
        else:
            topic_row.append(0)
    row = row + topic_row
    rows.append(row)
rows[0:10]

[['id',
  'text',
  'Ajoneuvot ja liikenne',
  'Koti ja rakentaminen',
  'Paikkakunnat',
  'Suhteet',
  'Terveys',
  'Yhteiskunta'],
 [0,
  'Suomen talous on kuralla ja siihen on yksi syyllinen: ammattiyhdistysliike. Vielä muutama vuosi sitten voimakasta ammattiyhdistysliikettä pidettiin suomalaisen yhteiskunnan vahvuutena. Ihailtiin konsensusta, sopimisen kulttuuria ja muuta työmarkkinanostalgiaa. Juuri päättyneellä eduskuntakaudella kaikki vaikuttaa kuitenkin muuttuneen. Ideologioista ja politiikasta vapaat talousviisaat syyttävät ay-liikettä loputtomasta palkkakirnuamisesta, elinkeinoelämä haukkuu ay-liikettä laittomista lakoista ja väistyvä pääministeri höyryää konsensuksen kuolemasta. Lisäksi pääkirjoituksissa ja talouslehdistössä surkutellaan, kuinka Kansallisen Selviämisen kannalta välttämättömiksi päätetyt rakenteelliset uudistukset, eli esimerkiksi heikennykset työttömyysturvaan, jäävät toteutumatta ay-liikkeen liiallisen vallan, konservatiivisuuden ja yleisen pahuuden vuoksi.

In [43]:
len(rows)

40048

In [44]:
writer = csv.writer(open("/tmp/suomi24.csv", 'w'))
for row in rows:
    writer.writerow(row)